# SIC-Centered Industry Classification Entity Resolution


## Purpose
This notebook focuses on entity resolution for industry classification lookup tables starting with SIC (Standard Industrial Classification) as the primary table. We'll use AI functions to standardize and match industry codes across three different classification systems:

### Target Tables
- **SIC** (Standard Industrial Classification) - PRIMARY/LEFT TABLE
- **MCC** (Merchant Category Codes)
- **NAICS** (North American Industry Classification System)  

### Approach
1. Load and explore each classification table
2. Create cross joins with SIC as the leftmost table
3. Apply AI_SIMILARITY to find matching classifications across systems
4. Use AI_CLASSIFY to identify best matches for each SIC code
5. Create unified SIC-centered industry mapping for entity resolution

### Expected Output
- One record for each SIC code
- Ability to lookup corresponding MCC and NAICS values for any SIC


In [1]:
# =============================================================================
# 🔧 ENTITY RESOLUTION FRAMEWORK
# =============================================================================

# 1. IMPORTS & SETUP
# Python Data 
import pandas as pd
from pydantic import BaseModel, Field
import json

# Python Formatting & Display
import humanize 
from datetime import datetime
from textwrap import dedent

#  Snowpark
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
import snowflake.snowpark.window as W
from snowflake.snowpark import Session
from snowflake.snowpark.context import get_active_session

# Cortex
import snowflake.cortex as C

# Helper Functions
def display_df_info(spdf, name="DataFrame"):
    """
    Display first 10 rows and metrics for a Snowpark DataFrame
    
    Args:
        spdf: Snowpark DataFrame to analyze
        name: Name to display for the DataFrame
    """
    # Get row and column counts
    row_count = spdf.count()
    col_count = len(spdf.columns)
    
    print(f"\n📊 {name} Overview:")
    print(f"  • Rows: {humanize.intword(row_count)} ({humanize.intcomma(row_count)})")
    print(f"  • Columns: {col_count}")
    
    print("\n🔍 First 10 rows:")
    spdf.limit(10).show()

def show_full_df(df, num_rows=10):
    """Display DataFrame with full formatting"""
    return df.limit(num_rows).to_pandas().style.set_properties(**{
        'text-align': 'left',
        'white-space': 'pre-wrap'
    }).set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

def clean_na_values(df, columns_to_select=None):
    """
    Replace 'NA' string values with None in all string columns of a dataframe
    
    Args:
        df: Snowpark DataFrame to clean
        columns_to_select: Optional list of columns to select in output DataFrame
        
    Returns:
        Snowpark DataFrame with 'NA' values replaced with None
    """
    # Get all string columns
    string_columns = [field.name for field in df.schema.fields 
                     if isinstance(field.datatype, T.StringType)]

    # Create list of column transformations
    column_transformations_list = [
        F.when(F.col(column) == 'NA', None)
         .when((F.col(column) == 'x') & (column == 'FEDTAXID'), None)
         .otherwise(F.col(column))
        for column in string_columns
    ]

    # Apply all transformations at once
    cleaned_df = df.with_columns(string_columns, column_transformations_list)
    
    # Select specified columns if provided
    if columns_to_select:
        cleaned_df = cleaned_df.select(columns_to_select)
        
    return cleaned_df

print("✅ All imports and helper functions loaded successfully")


/Users/jsoliz/src/GPN/entity_resolution/AISQLv1/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'validate_by_name'
  warnings.warn(message, UserWarning)


✅ All imports and helper functions loaded successfully


In [2]:
# 2. SESSION INITIALIZATION
def initialize_session():
    """Initialize Snowflake session with fallback options"""
    try:
        # First check for active session
        session = get_active_session()
        print("🔑 Using existing active Snowflake session")
        return session
    except Exception as e:
        print(f"⚠️  No active session found: {e}")
        try:
            # Try to load local credentials
            with open("/Users/jsoliz/.creds/gpn_connection.json", 'r') as f:
                connection_params = json.load(f)
            session = Session.builder.configs(connection_params).create()
            print("🔑 Local session initialized successfully")
            return session
        except Exception as e2:
            print(f"❌ Session initialization failed: {e2}")
            return None

# Initialize session
session = initialize_session()

if session:
    print(f"📊 Session active: {session.get_current_warehouse()}")
    print(f"🏢 Database: {session.get_current_database()}")
    print(f"📁 Schema: {session.get_current_schema()}")
else:
    print("❌ No session available - please check your connection configuration")


⚠️  No active session found: (1403): No default Session is found. Please create a session before you call function 'udf' or use decorator '@udf'.
Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/3a7077d2-14ae-4001-a736-75e0437e2b89/saml2?SAMLRequest=lZJBb%2BIwEIX%2FSuQ9J3ES0oAFVCyogMS2tMBq1ZtJJuDFsVmP05T99WtCkdpDK%2B3NGr9nfzNv%2BrevlfRewKDQakCigBIPVK4LoXYDslnf%2BV3ioeWq4FIrGJATILkd9pFX8shGtd2rJ%2FhTA1rPPaSQtRcDUhvFNEeBTPEKkNmcrUY%2FFiwOKDsabXWuJXln%2BdrBEcFYR3i1FCgc3t7aIwvDpmmCJgm02YUxpTSkvdCpzpJvV%2F2r6%2BkTfRTSzlnvFE6%2BfGP7LtRlBF9hbS8iZLP1eukvH1Zr4o2uqGOtsK7ArMC8iBw2T4sLADqCPXBjpRuq3%2Fy9SWMaB6h0U0p%2BgFxXx9q6VwN3CksoQql3wjU%2BnwzI8SCKh%2B34sJ%2BOykyZRz59%2FrWp64TGs8l29Ruz3rbqzk6LaTm2xZTmxPt5TTY%2BJztHrGGuznlaV6Jx6tPMT6I1TVnaYzQNulnvmXgTl6dQ3LbOK3TLEVQi

### Schemata (Confirm Tables)


In [3]:
# Load the classification tables
print("📥 Loading classification lookup tables...")

# Load all three classification systems
sic_lu_spdf = session.table("sandbox.javier.LU_SIC")
mcc_lu_spdf = session.table("sandbox.javier.LU_MCC").where(~F.col("MCC").startswith("3"))
naics_lu_spdf = session.table("sandbox.javier.LU_NAICS")

# Get record counts
sic_count = sic_lu_spdf.count()
mcc_count = mcc_lu_spdf.count() 
naics_count = naics_lu_spdf.count()

print(f"🏭 SIC Records (Primary): {sic_count:,} total")
print(f"🎯 MCC Records: {mcc_count:,} total")
print(f"📋 NAICS Records: {naics_count:,} total")

print(f"\n🔍 SIC Table Sample (Primary Table):")
show_full_df(sic_lu_spdf.limit(3))

print(f"\n🎯 MCC Table Sample:")
mcc_lu_spdf.limit(3).show()

print(f"\n📋 NAICS Table Sample:")  
naics_lu_spdf.limit(3).show()

# Get schema information
print("\n📊 Schema Summary:")
print("=" * 50)

print(f"\n🏭 SIC Schema (Primary - {len(sic_lu_spdf.schema.names)} columns):")
for field in sic_lu_spdf.schema.fields:
    print(f"  - {field.name}: {field.datatype}")

print(f"\n🎯 MCC Schema ({len(mcc_lu_spdf.schema.names)} columns):")
for field in mcc_lu_spdf.schema.fields:
    print(f"  - {field.name}: {field.datatype}")

print(f"\n📋 NAICS Schema ({len(naics_lu_spdf.schema.names)} columns):")
for field in naics_lu_spdf.schema.fields:
    print(f"  - {field.name}: {field.datatype}")


📥 Loading classification lookup tables...
🏭 SIC Records (Primary): 1,005 total
🎯 MCC Records: 286 total
📋 NAICS Records: 2,125 total

🔍 SIC Table Sample (Primary Table):

🎯 MCC Table Sample:
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"MCC"  |"MCC_DESCRIPTIVE_TITLE"                 |"INCLUDED_IN_THIS_MCC"                              |"SIMILAR_MCC_CODES"                                 |"SIMILAR_MCC_CODE_READABLE"                         |"MCC_ARRAY"                                         |"MCC_ARR_MAPS"                                      |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
# Create lookup tables with text descriptions for reuse (DRY principle)
print("🔧 Creating reusable lookup tables with text concatenations...")

# SIC lookup with concatenated text (Primary table)
sic_lu_with_text_spdf = sic_lu_spdf.select(
    "SIC_INDUSTRY_CODE",
    "SIC_INDUSTRY_DESCRIPTION",
    "SIC_MAJOR_GROUP_DESCRIPTION",
    "SIC_DIVISION_DESCRIPTION",
    F.concat(
        F.col("SIC_INDUSTRY_DESCRIPTION"),
        F.lit(" - "),
        F.col("SIC_MAJOR_GROUP_DESCRIPTION")
    ).alias("SIC_TEXT")
)

# MCC lookup with concatenated text
mcc_lu_with_text_spdf = mcc_lu_spdf.select(
    "MCC",
    "MCC_DESCRIPTIVE_TITLE",
    "INCLUDED_IN_THIS_MCC",
    F.concat(
        F.col("MCC_DESCRIPTIVE_TITLE"),
        F.lit(" - "),
        F.col("INCLUDED_IN_THIS_MCC")
    ).alias("MCC_TEXT")
)

# NAICS lookup with concatenated text
naics_lu_with_text_spdf = naics_lu_spdf.select(
    "CODE",
    "TITLE", 
    "DESCRIPTION",
    "DESCRIPTION_FULL",
    F.concat(
        F.col("TITLE"),
        F.lit(" - "),
        F.col("DESCRIPTION_FULL")
    ).alias("NAICS_TEXT")
)

print("✅ Reusable lookup tables with text concatenations created!")
print(f"🏭 SIC with text: {sic_lu_with_text_spdf.count():,} records")
print(f"🎯 MCC with text: {mcc_lu_with_text_spdf.count():,} records")
print(f"📋 NAICS with text: {naics_lu_with_text_spdf.count():,} records")


🔧 Creating reusable lookup tables with text concatenations...
✅ Reusable lookup tables with text concatenations created!
🏭 SIC with text: 1,005 records
🎯 MCC with text: 286 records
📋 NAICS with text: 2,125 records


# 🏭 SIC → MCC Mapping


## Objective
Create systematic mappings between SIC and MCC classification systems using Snowflake's AI functions.
**SIC is the primary/leftmost table** - we want one record per SIC code with corresponding MCC lookups.

## Step-by-Step Approach

### Step 1: Cross Join and AI Similarity Calculation
- **Goal**: Cross join SIC (left) with MCC (right) tables
- **AI Function**: Use `AI_SIMILARITY()` to compare names and descriptions between all SIC-MCC pairs
- **Output**: Every SIC record paired with every MCC record, including similarity scores

### Step 2: Ranking and Top 3 Selection  
- **Goal**: Use ranking function to keep top 3 MCC matches per SIC code
- **Method**: `ROW_NUMBER()` or `RANK()` partitioned by SIC, ordered by similarity score DESC
- **Output**: Maximum 3 MCC candidates per SIC record

### Step 3: AI Classification for Final Selection
- **Goal**: Use `AI_CLASSIFY()` to pick the best match among the top 3 candidates
- **Method**: Let AI choose the most appropriate MCC match for each SIC
- **Output**: Final `sic_mcc_lu_spdf` with one-to-one SIC→MCC mapping

---

## 🚀 Step 1: Cross Join and AI Similarity Calculation


In [5]:
# Step 1: Cross Join SIC and MCC with AI Similarity Calculation

print("\n🔍 First, let's examine the table structures:")
print("\nSIC Table Columns (Primary):")
print(sic_lu_with_text_spdf.columns)
print("\nMCC Table Columns:")
print(mcc_lu_with_text_spdf.columns)

print(f"\n📊 Table Sizes:")
print(f"SIC Records (Primary): {humanize.intcomma(sic_lu_with_text_spdf.count())}")
print(f"MCC Records: {humanize.intcomma(mcc_lu_with_text_spdf.count())}")
print(f"Cross Join Size: {humanize.intcomma(sic_lu_with_text_spdf.count() * mcc_lu_with_text_spdf.count())} total combinations")

# Create cross join with AI similarity calculation
print("\n🤖 Creating cross join with AI_SIMILARITY scores...")

import time
start_time = time.time()

# Time the cross join creation
sic_mcc_cross_join = (
    sic_lu_with_text_spdf.alias("sic").cross_join(
        mcc_lu_with_text_spdf.alias("mcc")
    )
)

# Time the column selection and AI similarity calculation
sic_mcc_cross_join = (
    sic_mcc_cross_join.select(
        # SIC fields (Primary)
        F.col("SIC_INDUSTRY_CODE").alias("SIC_CODE"),
        F.col("SIC_TEXT"),
        
        # MCC fields  
        F.col("MCC").alias("MCC_CODE"),
        F.col("MCC_TEXT"),
        
        # AI Similarity score
        F.call_function("AI_SIMILARITY",
            F.col("SIC_TEXT"),
            F.col("MCC_TEXT")
        ).alias("SIMILARITY_SCORE")
    )
)

# Write results to Snowflake table
print("\n💾 Writing results to Snowflake table...")
sic_mcc_cross_join.write.mode("overwrite").saveAsTable("sandbox.javier.sic_mcc_cross_join")

print("\n✅ Cross join with AI similarity created and saved successfully!")

end_time = time.time()
print(f"⏱️ Total execution time: {round(end_time - start_time, 2)} seconds")



🔍 First, let's examine the table structures:

SIC Table Columns (Primary):
['SIC_INDUSTRY_CODE', 'SIC_INDUSTRY_DESCRIPTION', 'SIC_MAJOR_GROUP_DESCRIPTION', 'SIC_DIVISION_DESCRIPTION', 'SIC_TEXT']

MCC Table Columns:
['MCC', 'MCC_DESCRIPTIVE_TITLE', 'INCLUDED_IN_THIS_MCC', 'MCC_TEXT']

📊 Table Sizes:
SIC Records (Primary): 1,005
MCC Records: 286


DataFrame.alias() is experimental since 1.5.0. Do not use it in production. 


Cross Join Size: 287,430 total combinations

🤖 Creating cross join with AI_SIMILARITY scores...

💾 Writing results to Snowflake table...

✅ Cross join with AI similarity created and saved successfully!
⏱️ Total execution time: 46.43 seconds


In [6]:
# Step 2: Load Cross Join Table and Apply Ranking for SIC-MCC

print("📂 Loading existing SIC-MCC cross join table...")
sic_mcc_cross_join = session.table("sandbox.javier.sic_mcc_cross_join")

print("🔍 Examining cross join table structure:")
print("Columns:", sic_mcc_cross_join.columns)
print(f"Total records: {humanize.intcomma(sic_mcc_cross_join.count())}")

print("\n🏆 Applying ranking to get top 3 MCC matches per SIC...")

# Apply ROW_NUMBER() window function to rank MCC matches by similarity score
sic_mcc_top_3_matches = sic_mcc_cross_join.select(
    "*",
    F.row_number().over(
        W.Window.partition_by("SIC_CODE").orderBy(F.col("SIMILARITY_SCORE").desc())
    ).alias("SIMILARITY_RANK")
).filter(
    F.col("SIMILARITY_RANK") <= 3
)

print("✅ Ranking applied! Keeping top 3 MCC matches per SIC code.")
print(f"📊 Reduced from {humanize.intcomma(sic_mcc_cross_join.count())} to {humanize.intcomma(sic_mcc_top_3_matches.count())} records")

# Write results to table for Step 3
sic_mcc_top_3_matches.write.mode("overwrite").save_as_table("sandbox.javier.sic_mcc_cross_join_top3")
print("\n💾 Results saved to table for SIC-MCC AI classification!")


📂 Loading existing SIC-MCC cross join table...
🔍 Examining cross join table structure:
Columns: ['SIC_CODE', 'SIC_TEXT', 'MCC_CODE', 'MCC_TEXT', 'SIMILARITY_SCORE']
Total records: 287,430

🏆 Applying ranking to get top 3 MCC matches per SIC...
✅ Ranking applied! Keeping top 3 MCC matches per SIC code.
📊 Reduced from 287,430 to 3,015 records

💾 Results saved to table for SIC-MCC AI classification!


In [7]:
# Step 3: AI Classification for Final SIC-MCC Selection

print("📂 Loading top 3 SIC-MCC results table...")
sic_mcc_cross_join_top3_spdf = session.table("sandbox.javier.sic_mcc_cross_join_top3")

print("🤖 Preparing data for AI_CLASSIFY to select best MCC match for each SIC...")

# Group the top 3 MCC codes per SIC into arrays for AI_CLASSIFY
sic_with_mcc_categories = sic_mcc_cross_join_top3_spdf.select(
    "SIC_CODE",
    "SIC_TEXT", 
    "MCC_CODE",
    "MCC_TEXT",
    "SIMILARITY_SCORE",
    "SIMILARITY_RANK"
).group_by(
    "SIC_CODE", "SIC_TEXT"
).agg(
    # Create array of MCC codes as classification categories
    F.array_agg(F.expr("MCC_CODE")).within_group("SIMILARITY_RANK").alias("MCC_CODES"),
    # Also keep MCC titles for reference
    F.array_agg(F.expr("MCC_TEXT")).within_group("SIMILARITY_RANK").alias("MCC_TEXTS"),
    F.array_agg(F.expr("SIMILARITY_RANK")).within_group("SIMILARITY_RANK").alias("SIMILARITY_RANKS")
)

print("✅ MCC categories prepared!")

print(f"\n🤖 Using AI_CLASSIFY to select best MCC matches for {humanize.intcomma(sic_with_mcc_categories.count())} SIC codes...")

# Use AI_CLASSIFY to classify each SIC into one of its 3 MCC categories
sic_mcc_classified = sic_with_mcc_categories.select(
    "SIC_CODE",
    "SIC_TEXT",
    "MCC_CODES",
    "MCC_TEXTS",
   
    # Use AI_CLASSIFY to select best MCC match from the 3 options
    F.call_function("AI_CLASSIFY",
        F.concat(F.col("SIC_CODE"), F.lit(". "), F.col("SIC_TEXT")),    # input
        F.col("MCC_TEXTS"),                                             # list_of_categories
    ).alias("AI_CLASSIFIED_MCC_TEXT")
)

# Save AI classifications
sic_mcc_classified.selectExpr("""SIC_CODE
                                , SIC_TEXT
                                , MCC_TEXTS
                                , AI_CLASSIFIED_MCC_TEXT:labels[0]::string                   as AI_CLASSIFIED_MCC_TEXT_STR
                                , COALESCE(AI_CLASSIFIED_MCC_TEXT_STR, MCC_TEXTS[0]::string) as MOST_LIKELY_MCC_TEXT
                                , AI_CLASSIFIED_MCC_TEXT_STR is not null as CLASSIFY_HAD_RETURN 
                                ,CLASSIFY_HAD_RETURN AND  AI_CLASSIFIED_MCC_TEXT_STR = MCC_TEXTS[0]::string as CLASSIFY_RETURNED_MOST_SIMILAR_MCC
                              """
).write.mode("overwrite").save_as_table("sandbox.javier.sic_mcc_classified")

print("✅ SIC-MCC AI classifications complete and saved!")


📂 Loading top 3 SIC-MCC results table...
🤖 Preparing data for AI_CLASSIFY to select best MCC match for each SIC...
✅ MCC categories prepared!

🤖 Using AI_CLASSIFY to select best MCC matches for 1,005 SIC codes...
✅ SIC-MCC AI classifications complete and saved!


In [8]:
# Create SIC to MCC Mapping Table

print("📥 Loading SIC-MCC AI classifications...")
sic_mcc_classified = session.table("sandbox.javier.sic_mcc_classified")

# Get distinct MCC code-text pairs from cross join table for joining back MCC codes
print("📊 Getting distinct MCC code-text pairs...")
mcc_pairs = (
    mcc_lu_with_text_spdf
    .select(
        F.col("MCC").alias("MCC_CODE"),
        F.col("MCC_TEXT")
    )
    .distinct()
)

# Join MCC codes back to AI classifications based on matched text
print("🔄 Joining MCC codes to AI classifications...")
sic_mcc_lookup = (
    sic_mcc_classified.join(
        mcc_pairs.distinct(),
        sic_mcc_classified["MOST_LIKELY_MCC_TEXT"] == mcc_pairs["MCC_TEXT"],
        "left"
    )
    .select(
        sic_mcc_classified["SIC_CODE"],
        mcc_pairs["MCC_CODE"].alias("MOST_LIKELY_MCC_CODE")
    )
)

# Create final SIC to MCC mapping table
code_map_sic_mcc = sic_mcc_lookup.select(
    "SIC_CODE",
    "MOST_LIKELY_MCC_CODE"
).distinct().orderBy("SIC_CODE")

code_map_sic_mcc.write.mode("overwrite").save_as_table("sandbox.javier.code_map_sic_mcc")

print(f"✅ SIC to MCC mapping table created!")
print(f"📊 {humanize.intcomma(code_map_sic_mcc.count())} SIC codes mapped to MCC")


📥 Loading SIC-MCC AI classifications...
📊 Getting distinct MCC code-text pairs...
🔄 Joining MCC codes to AI classifications...
✅ SIC to MCC mapping table created!
📊 1,005 SIC codes mapped to MCC


In [9]:
# Verify SIC to MCC mapping is 1:1
print("\n🔍 Verifying SIC to MCC mapping cardinality...")

sic_mcc_counts = (
    sic_mcc_classified
    .groupBy("SIC_CODE")
    .agg(F.count("*").alias("MCC_COUNT"))
    .where(F.col("MCC_COUNT") > 1)
)

multiple_mappings_count = sic_mcc_counts.count()

if multiple_mappings_count > 0:
    print(f"⚠️ Found {multiple_mappings_count} SIC codes with multiple MCC mappings:")
    sic_mcc_counts.show()
else:
    print("✅ Verified: Each SIC code maps to exactly one MCC code")

# Create histogram of MCC mappings per SIC code from classified data
print("\n📊 Distribution of MCC mappings per SIC code:")

mapping_distribution = (
    sic_mcc_classified
    .groupBy("SIC_CODE")
    .agg(F.count("*").alias("MCC_COUNT"))
    .groupBy("MCC_COUNT")
    .agg(F.count("*").alias("SIC_COUNT"))
    .orderBy("MCC_COUNT")
)

mapping_distribution.show()

# Compare number of SIC codes in mapping vs source table
print("\n🔍 Comparing SIC code coverage...")

mapped_sic_count = code_map_sic_mcc.select("SIC_CODE").distinct().count()
source_sic_count = sic_lu_with_text_spdf.select("SIC_INDUSTRY_CODE").distinct().count()

print(f"Source SIC codes: {humanize.intcomma(source_sic_count)}")
print(f"Mapped SIC codes: {humanize.intcomma(mapped_sic_count)}")

if mapped_sic_count < source_sic_count:
    print(f"⚠️ Missing mappings for {humanize.intcomma(source_sic_count - mapped_sic_count)} SIC codes")
    
    # Show unmapped SIC codes
    print("\nUnmapped SIC codes:")
    unmapped_sic = (
        sic_lu_with_text_spdf.select("SIC_INDUSTRY_CODE", "SIC_INDUSTRY_DESCRIPTION")
        .join(code_map_sic_mcc, sic_lu_with_text_spdf.SIC_INDUSTRY_CODE == code_map_sic_mcc.SIC_CODE, "left_anti")
    )
    unmapped_sic.show()
else:
    print("✅ All source SIC codes have been mapped")



🔍 Verifying SIC to MCC mapping cardinality...
✅ Verified: Each SIC code maps to exactly one MCC code

📊 Distribution of MCC mappings per SIC code:
-----------------------------
|"MCC_COUNT"  |"SIC_COUNT"  |
-----------------------------
|1            |1005         |
-----------------------------


🔍 Comparing SIC code coverage...
Source SIC codes: 1,005
Mapped SIC codes: 1,005
✅ All source SIC codes have been mapped


# 🏭 SIC → NAICS Mapping

## Objective
Create systematic mappings between SIC and NAICS classification systems using Snowflake's AI functions.
**SIC is the primary/leftmost table** - we want one record per SIC code with corresponding NAICS lookups.

## Step-by-Step Approach
Following the same methodology as SIC-MCC mapping:

1. **Cross Join and AI Similarity**: SIC (left) with NAICS (right) 
2. **Ranking and Top 3 Selection**: Keep best NAICS matches per SIC
3. **AI Classification**: Use AI_CLASSIFY for final selection

---


In [10]:
# SIC-NAICS Mapping Implementation (Following Same Pattern as SIC-MCC)

print("📋 Creating SIC-NAICS mappings using the same methodology...")

# Step 1: Cross Join SIC and NAICS with AI Similarity
print("\n📊 Step 1: Creating SIC-NAICS cross join with AI_SIMILARITY...")

sic_naics_cross_join = (
    sic_lu_with_text_spdf.alias("sic").cross_join(naics_lu_with_text_spdf.alias("naics"))
    .select(
        # SIC fields (Primary)
        F.col("SIC_INDUSTRY_CODE").alias("SIC_CODE"),
        F.col("SIC_TEXT"),
        # NAICS fields  
        F.col("CODE").alias("NAICS_CODE"),
        F.col("NAICS_TEXT"),
        # AI Similarity score
        F.call_function("AI_SIMILARITY", F.col("SIC_TEXT"), F.col("NAICS_TEXT")).alias("SIMILARITY_SCORE")
    )
)

sic_naics_cross_join.write.mode("overwrite").saveAsTable("sandbox.javier.sic_naics_cross_join")
print("✅ SIC-NAICS cross join created!")

# Step 2: Ranking - Top 3 NAICS matches per SIC
print("\n🏆 Step 2: Applying ranking to get top 3 NAICS matches per SIC...")

sic_naics_top_3 = sic_naics_cross_join.select(
    "*",
    F.row_number().over(W.Window.partition_by("SIC_CODE").orderBy(F.col("SIMILARITY_SCORE").desc())).alias("SIMILARITY_RANK")
).filter(F.col("SIMILARITY_RANK") <= 3)

sic_naics_top_3.write.mode("overwrite").save_as_table("sandbox.javier.sic_naics_cross_join_top3")
print("✅ Top 3 NAICS matches per SIC saved!")


# Step 3: AI Classification
print("\n🤖 Step 3: Using AI_CLASSIFY for final NAICS selection...")

# Group top 3 NAICS options for AI_CLASSIFY
sic_with_naics_categories = sic_naics_top_3.group_by("SIC_CODE", "SIC_TEXT").agg(
    F.array_agg(F.expr("NAICS_TEXT")).within_group("SIMILARITY_RANK").alias("NAICS_TEXTS")
)

# Apply AI_CLASSIFY
sic_naics_classified = sic_with_naics_categories.select(
    "SIC_CODE", "SIC_TEXT", "NAICS_TEXTS",
    F.call_function("AI_CLASSIFY", F.concat(F.col("SIC_CODE"), F.lit(". "), F.col("SIC_TEXT")), F.col("NAICS_TEXTS")).alias("AI_CLASSIFIED_NAICS_TEXT")
)


📋 Creating SIC-NAICS mappings using the same methodology...

📊 Step 1: Creating SIC-NAICS cross join with AI_SIMILARITY...
✅ SIC-NAICS cross join created!

🏆 Step 2: Applying ranking to get top 3 NAICS matches per SIC...
✅ Top 3 NAICS matches per SIC saved!

🤖 Step 3: Using AI_CLASSIFY for final NAICS selection...


In [11]:
# Save and create mapping table
sic_naics_classified.selectExpr("""SIC_CODE, SIC_TEXT, NAICS_TEXTS,
                               AI_CLASSIFIED_NAICS_TEXT:labels[0]::string as AI_CLASSIFIED_NAICS_TEXT_STR,
                               COALESCE(AI_CLASSIFIED_NAICS_TEXT_STR, NAICS_TEXTS[0]::string) as MOST_LIKELY_NAICS_TEXT"""
).write.mode("overwrite").save_as_table("sandbox.javier.sic_naics_classified")

session.table("sandbox.javier.sic_naics_classified").print_schema()
# Create final mapping table
naics_pairs = session.table("sandbox.javier.sic_naics_cross_join").select("NAICS_CODE", "NAICS_TEXT").distinct()
sic_naics_lookup = (
    session.table("sandbox.javier.sic_naics_classified").join(naics_pairs, F.col("MOST_LIKELY_NAICS_TEXT") == F.col("NAICS_TEXT"), "left")
    .select("SIC_CODE", F.col("NAICS_CODE").alias("MOST_LIKELY_NAICS_CODE")).distinct().orderBy("SIC_CODE")
)
sic_naics_lookup.write.mode("overwrite").save_as_table("sandbox.javier.code_map_sic_naics")

print("✅ SIC-NAICS mapping complete!")
print(f"📊 {humanize.intcomma(sic_naics_lookup.count())} SIC codes mapped to NAICS")


root
 |-- "SIC_CODE": StringType(15) (nullable = True)
 |-- "SIC_TEXT": StringType(503) (nullable = True)
 |-- "NAICS_TEXTS": ArrayType (nullable = False)
 |   |-- element: StringType()
 |-- "AI_CLASSIFIED_NAICS_TEXT_STR": StringType() (nullable = True)
 |-- "MOST_LIKELY_NAICS_TEXT": StringType() (nullable = True)
✅ SIC-NAICS mapping complete!
📊 1,302 SIC codes mapped to NAICS


# 🏭 Final SIC-Centered Industry Classification Table

## Objective
Create the final SIC-centered industry classification table by joining all mappings together.
This will provide one record per SIC code with corresponding MCC and NAICS lookups.

## Final Output Structure
- **SIC_CODE** (Primary Key)
- **MCC_CODE** (Mapped)
- **NAICS_CODE** (Mapped)
- Plus descriptive fields from all three classification systems

---


In [12]:
# Join SIC-MCC and SIC-NAICS lookups to create final classification table
sic_mcc = session.table("sandbox.javier.code_map_sic_mcc").select(
    F.col("SIC_CODE").alias("SIC_CODE_MCC_SIDE"),
    "MOST_LIKELY_MCC_CODE"
)
sic_naics = session.table("sandbox.javier.code_map_sic_naics").select(
    F.col("SIC_CODE").alias("SIC_CODE_NAICS_SIDE"), 
    "MOST_LIKELY_NAICS_CODE"
)

# Create final classification table with all descriptive fields
sic_classifications = (
    sic_mcc
    .join(sic_naics, sic_mcc.SIC_CODE_MCC_SIDE == sic_naics.SIC_CODE_NAICS_SIDE, "full")
    .join(sic_lu_with_text_spdf, F.coalesce(F.col("SIC_CODE_MCC_SIDE"), F.col("SIC_CODE_NAICS_SIDE")) == sic_lu_with_text_spdf.SIC_INDUSTRY_CODE, "left")
    .join(
        mcc_lu_with_text_spdf.select("MCC", F.col("MCC_DESCRIPTIVE_TITLE").alias("MCC_TEXT")), 
        F.col("MOST_LIKELY_MCC_CODE") == mcc_lu_with_text_spdf.MCC,
        "left"
    )
    .join(
        naics_lu_with_text_spdf.select("CODE", F.col("TITLE").alias("NAICS_TEXT")),
        F.col("MOST_LIKELY_NAICS_CODE") == naics_lu_with_text_spdf.CODE,
        "left"
    )
    .select(
        F.coalesce(F.col("SIC_CODE_MCC_SIDE"), F.col("SIC_CODE_NAICS_SIDE")).alias("SIC_CODE"),
        F.col("SIC_INDUSTRY_DESCRIPTION").alias("SIC_TEXT"),
        F.col("MOST_LIKELY_MCC_CODE").alias("MCC_CODE"),
        "MCC_TEXT", 
        F.col("MOST_LIKELY_NAICS_CODE").alias("NAICS_CODE"),
        "NAICS_TEXT"
    )
    .orderBy("SIC_CODE")
)

# Save final classification table
sic_classifications.write.mode("overwrite").save_as_table("sandbox.javier.sic_industry_classifications")


In [13]:
# Final Statistics and Verification

print("📈 Final Statistics and Verification")
print("=" * 50)


# Verify one record per SIC code
sic_classifications = session.table("sandbox.javier.sic_industry_classifications")

# Get comprehensive statistics
total_sic_records = sic_classifications.count()
records_with_mcc = sic_classifications.filter(F.col("MCC_CODE").isNotNull()).count()
records_with_naics = sic_classifications.filter(F.col("NAICS_CODE").isNotNull()).count()
records_with_both = sic_classifications.filter(
    F.col("MCC_CODE").isNotNull() & F.col("NAICS_CODE").isNotNull()
).count()

print(f"\n🏭 SIC-Centered Classification Summary:")
print(f"Total SIC records:                   {total_sic_records:>6}")
print(f"SIC records with MCC mapping:        {records_with_mcc:>6} ({(records_with_mcc/total_sic_records*100):>5.1f}%)")
print(f"SIC records with NAICS mapping:      {records_with_naics:>6} ({(records_with_naics/total_sic_records*100):>5.1f}%)")
print(f"SIC records with both mappings:      {records_with_both:>6} ({(records_with_both/total_sic_records*100):>5.1f}%)")

# Verify exactly one record per SIC code
print(f"\n🔍 Verification - One Record Per SIC Code:")
records_per_sic = sic_classifications.group_by("SIC_CODE").count()
max_records_per_sic = records_per_sic.agg(F.max("count")).collect()[0][0]
min_records_per_sic = records_per_sic.agg(F.min("count")).collect()[0][0]

print(f"Max records per SIC: {max_records_per_sic}")
print(f"Min records per SIC: {min_records_per_sic}")

if max_records_per_sic == 1 and min_records_per_sic == 1:
    print("✅ SUCCESS: Exactly one record per SIC code confirmed!")
else:
    print("⚠️  WARNING: Multiple records found for some SIC codes")

# Show unique counts
unique_mcc = sic_classifications.select("MCC_CODE").distinct().count()
unique_naics = sic_classifications.select("NAICS_CODE").distinct().count()

print(f"\n📊 Unique Code Counts:")
print(f"Unique SIC codes:                    {total_sic_records:>6}")
print(f"Unique MCC codes mapped:             {unique_mcc:>6}")
print(f"Unique NAICS codes mapped:           {unique_naics:>6}")

print(f"\n✅ SIC-centered industry classification lookup complete!")
print(f"💡 Table: 'sic_industry_classifications' - One record per SIC with MCC and NAICS lookups")


📈 Final Statistics and Verification

🏭 SIC-Centered Classification Summary:
Total SIC records:                     1302
SIC records with MCC mapping:          1302 (100.0%)
SIC records with NAICS mapping:        1302 (100.0%)
SIC records with both mappings:        1302 (100.0%)

🔍 Verification - One Record Per SIC Code:
Max records per SIC: 2
Min records per SIC: 1
⚠️  WARNING: Multiple records found for some SIC codes

📊 Unique Code Counts:
Unique SIC codes:                      1302
Unique MCC codes mapped:                217
Unique NAICS codes mapped:              909

✅ SIC-centered industry classification lookup complete!
💡 Table: 'sic_industry_classifications' - One record per SIC with MCC and NAICS lookups


In [14]:
# Create comprehensive industry classification table with descriptions

print("🔄 Creating comprehensive SIC-centered industry classification table...")

# NOTE: This assumes the mapping tables code_map_sic_mcc and code_map_sic_naics have been created
# by following the same methodology as the original notebook but with SIC as the primary table

try:
    # Load final mappings (these would be created by the full implementation)
    code_map_sic_mcc = session.table("sandbox.javier.code_map_sic_mcc")
    code_map_sic_naics = session.table("sandbox.javier.code_map_sic_naics")
    
    # Join the SIC-MCC and SIC-NAICS mappings
    sic_industry_mappings = code_map_sic_mcc.join(
        code_map_sic_naics,
        on="SIC_CODE", 
        how="left"
    ).orderBy("SIC_CODE")

    # Join with lookup tables to get comprehensive descriptions
    sic_industry_classifications = \
    (sic_industry_mappings
        .join(sic_lu_spdf, sic_industry_mappings.SIC_CODE == sic_lu_spdf.SIC_INDUSTRY_CODE, "left")
        .join(mcc_lu_spdf, sic_industry_mappings.MOST_LIKELY_MCC_CODE == mcc_lu_spdf.MCC, "left") 
        .join(naics_lu_spdf, sic_industry_mappings.MOST_LIKELY_NAICS_CODE == naics_lu_spdf.CODE, "left")
        .selectExpr(
            """ SIC_CODE,
                MOST_LIKELY_MCC_CODE AS MCC_CODE,
                MOST_LIKELY_NAICS_CODE AS NAICS_CODE,
                -- SIC fields (Primary)
                SIC_INDUSTRY_DESCRIPTION AS SIC_TITLE,
                SIC_MAJOR_GROUP_DESCRIPTION AS SIC_MAJOR_GROUP_TITLE,
                SIC_DIVISION_DESCRIPTION AS SIC_DIVISION_TITLE,
                -- MCC fields  
                MCC_DESCRIPTIVE_TITLE AS MCC_TITLE,
                INCLUDED_IN_THIS_MCC AS MCC_INCLUDED_MERCHANTS,
                -- NAICS fields
                TITLE AS NAICS_TITLE,
                DESCRIPTION_FULL AS NAICS_DESCRIPTION
            """
        )
    )

    print("🔍 Sample of SIC-centered classification table:")
    sic_industry_classifications.limit(5).show()

    # Save the comprehensive table
    print(f"\n💾 Writing comprehensive classifications to 'sic_industry_classifications' table...")
    sic_industry_classifications.orderBy("SIC_CODE").write.mode("overwrite").saveAsTable("sandbox.javier.sic_industry_classifications")

    print(f"\n📊 Final table contains {humanize.intcomma(sic_industry_classifications.count())} SIC records")
    print("\n🏭 Final SIC-centered industry classification table created!")
    print("💡 You can now lookup MCC and NAICS values for any SIC code using this table")

except Exception as e:
    print(f"⚠️ Mapping tables not yet created. Complete the full implementation first.")
    print(f"   This cell shows the expected final outcome structure.")
    print(f"   Error: {e}")
    
    # Show expected structure with sample data
    print(f"\n📋 Expected Final Table Structure:")
    print("SIC_CODE | MCC_CODE | NAICS_CODE | SIC_TITLE | MCC_TITLE | NAICS_TITLE")
    print("---------|----------|------------|-----------|-----------|------------")
    print("5411     | 5411     | 445110     | Grocery   | Grocery   | Supermarket")
    print("5812     | 5812     | 722511     | Restaurant| Restaurant| Full-Service")
    print("...")
    print("\n🏭 Each SIC code will have exactly one record with corresponding MCC and NAICS lookups")


🔄 Creating comprehensive SIC-centered industry classification table...
🔍 Sample of SIC-centered classification table:
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SIC_CODE"  |"MCC_CODE"  |"NAICS_CODE"  |"SIC_TITLE"         |"SIC_MAJOR_GROUP_TITLE"            |"SIC_DIVISION_TITLE"              |"MCC_TITLE"                 |"MCC_INCLUDED_MERCHANTS"                            |"NAICS_TITLE"                      |"NAICS_DESCRIPTION"                                 |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [15]:
# Reload and display the saved table
print("\n📋 Displaying saved SIC industry classifications ordered by SIC code:")
sic_industry_classifications = session.table("sandbox.javier.sic_industry_classifications")
sic_industry_classifications.orderBy("SIC_CODE").show()
# Count distinct SIC codes
distinct_sic_count = sic_industry_classifications.select("SIC_CODE").distinct().count()
print(f"\n🔢 Total distinct SIC codes: {humanize.intcomma(distinct_sic_count)}")



📋 Displaying saved SIC industry classifications ordered by SIC code:
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SIC_CODE"  |"MCC_CODE"  |"NAICS_CODE"  |"SIC_TITLE"                           |"SIC_MAJOR_GROUP_TITLE"        |"SIC_DIVISION_TITLE"              |"MCC_TITLE"                 |"MCC_INCLUDED_MERCHANTS"                            |"NAICS_TITLE"    |"NAICS_DESCRIPTION"                                 |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0111        |763       

In [16]:
# Close session
session.close()
